In [2]:
import os

In [3]:
%pwd

'r:\\Projects\\Flight_Price_Prediction_ML\\Flight-Price-Prediction-ML-model\\research'

In [4]:
os.chdir('r:\\Projects\\Flight_Price_Prediction_ML\\Flight-Price-Prediction-ML-model')

In [5]:
%pwd

'r:\\Projects\\Flight_Price_Prediction_ML\\Flight-Price-Prediction-ML-model'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path:Path
    label_encoders:dict

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__ (
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH,
            label_encoding_filepath = LABEL_ENCODING_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)
            self.labels = read_yaml(label_encoding_filepath)

            create_directories([self.config.artifacts_root])


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        label_encoders = self.labels

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
             root_dir= config.root_dir,
             data_path=config.data_path,
             label_encoders = label_encoders
        )

        return data_transformation_config

In [8]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


In [9]:
obj = ConfigurationManager()


[2024-07-14 01:28:13,796: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-14 01:28:13,801: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-14 01:28:13,815: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-14 01:28:13,839: INFO: common: yaml file: label_encoding.yaml loaded successfully]
[2024-07-14 01:28:13,849: INFO: common: created directory at: artifacts]


In [10]:
list(obj.labels.airlines.keys())
df = pd.read_excel('research\\Data.xlsx')

In [11]:
a = list(obj.labels.airlines.keys())
a.pop()

'Vistara Premium economy'

In [12]:
for i in list(df.Airline.unique()):
    if i not in a:
        print("Mismatch")
        break
else:
    print("Match")


Mismatch


In [20]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config = config
        

    def data_transformation(self):
        df_ = pd.read_excel(self.config.data_path)

        df_ = df_.dropna()

        #creating columns for month,year,day of journey

        df_.Date_of_Journey = pd.to_datetime(df_.Date_of_Journey)
        df_['Journey_month'] = df_.Date_of_Journey.dt.month
        df_['Journey_year'] = df_.Date_of_Journey.dt.year
        df_['Journey_day'] = df_.Date_of_Journey.dt.day


        # creating columns for hours,min for depature and arrival time
        df_.Dep_Time = pd.to_datetime(df_.Dep_Time)
        df_['Dep_Time_hours'] = df_.Dep_Time.dt.hour
        df_['Dep_Time_mins'] = df_.Dep_Time.dt.minute
        df_.Arrival_Time = pd.to_datetime(df_.Arrival_Time)
        df_['Arrival_Time_hours'] = df_.Arrival_Time.dt.hour
        df_['Arrival_Time_mins'] = df_.Arrival_Time.dt.minute


        df_ = df_.drop(['Date_of_Journey','Dep_Time','Arrival_Time'],axis=1)

        def duration_hours(x):
            if 'h' in x:
                return int(x.split('h')[0])
            else:
                return 0


        def duration_mins(x):
            if 'h' in x and 'm' in x:
                return int(x.split(' ')[1].split('m')[0])

            elif 'm' in x:
                return int(x.split('m')[0])

            else:
                return 0


        df_['Duration_hours'] = df_['Duration'].apply(duration_hours)
        df_['Duration_minutes'] = df_['Duration'].apply(duration_mins)


        def total_mins(x):
            total = x['Duration_hours']*60 + x['Duration_minutes']
            return total

        df_['Duration_total_minutes'] = df_.apply(total_mins,axis=1)

        df_ = df_.drop('Duration',axis=1)

        sources = df_.Source.unique()


        # Applying One-Hot encoder in sources column
        for source in sources:
            df_['Source_'+source] = np.nan
            df_['Source_'+source] = np.where(df_['Source']==source,1,0)


        df_['Airline label'] = np.nan

        airlines = dict(self.config.label_encoders.airlines)
        
        # Applying Label encoding in airline column
        df_['Airline label'] = df_['Airline'].map(airlines)
        df_ = df_.drop('Airline',axis=1)


        destinations = dict(self.config.label_encoders.destinations)

        df_.Destination = df_.Destination.replace('Delhi','New Delhi')

        # Applying Label encoding in destination column
        df_['Destination label'] = df_.Destination.map(destinations)

        # Dropping unncessary columns
        df_ = df_.drop(['Source','Destination','Additional_Info'],axis=1)

        stops_labels = {'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4}

        # Applying label encoding in total_stops column
        df_['Total_Stops_label'] = df_['Total_Stops'].map(stops_labels)

        df_['Total_Stops_label'] = df_['Total_Stops_label'].astype('int')

        df_ = df_.drop('Total_Stops',axis=1)

        df_ = df_.drop('Route',axis=1)

        df_ = df_.drop('Journey_year',axis=1)   # only 2019 year exists

        # changing float values to int
        for col in df_.columns:
            if df_[col].dtype == 'float64':
                df_[col]= df_[col].astype('int')

        # Removing outliers

        #Calculate Q1 (25th percentile) and Q3 (75th percentile)

        Q1 = df_['Price'].quantile(0.25)
        Q3 = df_['Price'].quantile(0.75)

        # Calculate IQR
        IQR = Q3 - Q1

        # Define outlier bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Identify outliers
        outliers = df_[(df_['Price'] < lower_bound) | (df_['Price'] > upper_bound)]


        #Replacing outliers with median value

        df_['Price'] = np.where(df_['Price']>upper_bound,df_['Price'].median(),df_['Price'])


        return df_
    



    def train_test_spliting(self,data):
        
        train,test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir,'train.csv'),index=False)
        test.to_csv(os.path.join(self.config.root_dir,'test.csv'),index=False)

        logger.info("Splitted data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)



In [21]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data =  data_transformation.data_transformation()
    data_transformation.train_test_spliting(data)

except Exception as e:
    raise e

[2024-07-14 01:40:19,223: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-14 01:40:19,223: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-14 01:40:19,247: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-14 01:40:19,264: INFO: common: yaml file: label_encoding.yaml loaded successfully]
[2024-07-14 01:40:19,268: INFO: common: created directory at: artifacts]
[2024-07-14 01:40:19,271: INFO: common: created directory at: artifacts/data_transformation]


C:\Users\rithi\AppData\Local\Temp\ipykernel_25188\3834836195.py:13: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_.Date_of_Journey = pd.to_datetime(df_.Date_of_Journey)
C:\Users\rithi\AppData\Local\Temp\ipykernel_25188\3834836195.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_.Dep_Time = pd.to_datetime(df_.Dep_Time)
C:\Users\rithi\AppData\Local\Temp\ipykernel_25188\3834836195.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_.Arrival_Time = pd.to_datetime(df_.Arrival_Time)


[2024-07-14 01:40:26,107: INFO: 3834836195: Splitted data into training and test sets]
[2024-07-14 01:40:26,110: INFO: 3834836195: (8011, 18)]
[2024-07-14 01:40:26,113: INFO: 3834836195: (2671, 18)]
(8011, 18)
(2671, 18)
